In [14]:

%pip install dspy-ai

%pip install ipywidgets

%pip install python-dotenv

%pip install chromadb 
%pip install pymupdf

%pip install torch --index-url https://download.pytorch.org/whl/cu118

%pip install bitsandbytes

%pip install peft

%pip install accelerate

%pip install datasets

%pip install transformers





huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


RAG Model with prompt refinement

In [2]:
import torch
 
cuda_available = torch.cuda.is_available()
print(cuda_available)

True


In [3]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [4]:
import dspy
from openai import OpenAI
from dspy.retrieve.chromadb_rm import ChromadbRM
from chromadb.utils import embedding_functions
import chromadb

In [5]:
client = chromadb.PersistentClient(path="chromadb/cache")
collection = client.delete_collection("standard_full")
collection = client.get_or_create_collection(name="standard_full")

In [6]:
model = dspy.OpenAI(model = 'gpt-3.5-turbo-16k')
rm = ChromadbRM(collection_name='standard_full', persist_directory="chromadb/cache", embedding_function=embedding_functions.DefaultEmbeddingFunction())
dspy.settings.configure(lm=model, rm=rm)

In [7]:
def add_paragraphs_to_chromadb(paragraphs):
    documents = paragraphs
    ids = [f"id{i}" for i in range(len(paragraphs))]
    collection.add(documents=documents, ids=ids)

def read_txt_paragraphs(txt_file):
    paragraphs = []
    with open(txt_file) as info:
        lines = info.read()
        paragraphs = lines.split('\n\n')
    return paragraphs

txt_path = "chromadb/ngac_config_notes(updated).txt" 
txt_paragraphs = read_txt_paragraphs(txt_path)

paragraphs = txt_paragraphs 

print(paragraphs)
print(len(paragraphs))
add_paragraphs_to_chromadb(paragraphs)    

['\nThe NGAC security model defines the conceptual framework for the specification of a wide range of access control policies. The security model is intended to facilitate the understanding and analysis of the properties afforded to protect resources. The security model is defined formally in terms of the mathematics of set theory and predicate calculus in consonance with the Z notation (see ZNOT). The formal specification allows the essential properties for the composition and working of NGAC to be statedfree from constraints on how the properties are achieved.', '\nThe abstractions specified by the security model govern the operation of the NGAC Functional Architecture. The abstractions involve the elements, relations, and commands that are used to represent\nsecurity policies and provide the basis for rendering access decisions. The types of abstractions that are defined herein are as follows:\na) basic elements;\nb) relations;\nc) administrative commands; and\nd) access adjudicatio

In [8]:
import csv

example_dataset = "chromadb/NGACdata.csv"

with open(example_dataset, 'r') as file:
    dataset=csv.reader(file)

    descriptions=[]
    configurations=[]

    for example in dataset:
        if example[0] == 'policy':
            continue
        else:
            descriptions.append(example[0])
            configurations.append(example[1])

data = []

for i in range(len(descriptions)):
    data.append(dspy.Example(description= descriptions[i], configuration = configurations[i]).with_inputs("description"))



In [9]:
trainset = data[1:3]
devset = data[3:6]
testset = data[6:]

Signatures

In [10]:
class PromptOptimzer(dspy.Signature):
    ''' Rewrite the given access control policy description to be easily understood while maintaining all information and logic. '''
    configuration_description = dspy.InputField(desc=''' Information regarding organizational levels of access control that must strictly followed. 
                                        Note possible sets of individual users/objects and their levels of access/membership of groups. ''')                           
    optimized_paragraph  = dspy.OutputField()        

In [11]:
class PML_Generator(dspy.Signature):
  '''Generate the following access control policy configuration for the Policy Class using Policy Machine Language (PML). Ensure to create all provided users and user attributes by name encapsulate their access rights.
  Policy Machine Language(PML) statement format : 
                            create policy class "policyclass1"; - creates policy class "policyclass1" for all elements to be contained in - usually incapsulates all the elements of the policy;
                            create user attribute "userattribute1" in ["pc1"]; - creates user attribute "userattribute1" in policy class "pc1";
                            create object attribute "objectattribute1" in ["pc1"]; - creates object attribute "objectattribute1" in policy class "pc1";
                            assign "userattribute1" to ["userattribute2"]; - assigns "userattribute1" to "userattribute2". The permissions of "userattribute2" are inherited by "userattribute1";
                            create user "user1" in ["userattribute1" "userattribute2"]; - creates user "user1" with attributes "userattribute1" and "userattribute2";
                            associate "userattribute1" and "objectattribute1" with ["accessright1" "accessright2"]; - permits any user with "userattribute1" access "accessright1" and "accessright2" on "objectattribute1";
                                        '''

  natural_language_description = dspy.InputField(desc="Natural Language description of target access control policy")
  context = dspy.InputField(desc="Information relevant to access control policies")
  PML_statements = dspy.OutputField(desc = "Policy Machine Language (PML)")
    

Pipeline

In [40]:
class RAG(dspy.Module):
    def __init__(self):
        super().__init__()
        self.refine_prompt = dspy.ChainOfThought(PromptOptimzer)
        self.generate_answer = dspy.Predict(PML_Generator,max_tokens = 5000)
        self.retrieve = dspy.Retrieve(k=3)


    def forward(self, description):

        refined_prompt = self.refine_prompt(configuration_description=description) #Refines prompt
        context = self.retrieve(description).passages
        configurationPML = self.generate_answer(natural_language_description = refined_prompt.optimized_paragraph, 
                                                context=context) #Generates PML
        
        
        return dspy.Prediction(context=context, configuration= configurationPML) 
    

In [41]:
gpms_example = ''' On college campuses, only particular faculty members are eligible to create a proposal, called PDSWhole.
Research Faculty, Tenured Track Faculty, and Tenured Faculty can create proposals and are eligible to be PI, CoPI, and Senior Personnel of the proposal.
Adjunct Faculties can only be Senior Personnel, Clinical and Teaching faculty can be CoPI.
'''

In [42]:
rag_model = RAG()

In [43]:
model.inspect_history(n=1)




Parse the rating from a string.

---

Follow the following format.

Raw Rating Response: The string that contains the rating in it.
Rating: An integer valued rating.

---

Raw Rating Response: 1
Rating: 1





'\n\n\nParse the rating from a string.\n\n---\n\nFollow the following format.\n\nRaw Rating Response: The string that contains the rating in it.\nRating: An integer valued rating.\n\n---\n\nRaw Rating Response: 1\nRating:\x1b 1\x1b\n\n\n'

Metrics and Evaluation

In [44]:
class Evaluator(dspy.Signature):
    '''Evaluate the quality of an access control policy configuration according to a given criterion. Give a numbered rating.'''
    criterion = dspy.InputField(desc="The evaluation criterion")
    natural_language_description = dspy.InputField(desc="Description of target access control policy configuration.")
    gold_configuration = dspy.InputField(desc="An expert written target configuration generated in accordance to the description")
    generated_configuration = dspy.InputField(desc="The system's generated configuration in accordance to the description.")
    
    feedback = dspy.OutputField(desc="A rating between 1 and 5. IMPORTANT!! Only output the rating as an `int` and nothing else.")


In [45]:
class RatingParser(dspy.Signature):
    """Parse the rating from a string."""

    raw_rating_response = dspy.InputField(desc="The string that contains the rating in it.")
    rating = dspy.OutputField(desc="An integer valued rating.")

In [46]:
class RAGMetric(dspy.Module):
    def __init__(self):
        self.evaluator = dspy.ChainOfThought(Evaluator)
        self.rating_parser = dspy.Predict(RatingParser)


    def forward(self, gold, pred, trace=None):
        description = str(gold.description)
        gold_config = str(gold.configuration)
        gen_config = str(pred.configuration)

        users = "Does the generated configuration include all user and object attributes?"
        ground_truth = f"The Ground Truth configuration in accordance to the policy: {description} is given as: \n \n {gold_config} \n \n How aligned is this Predicted Answer? {gen_config}"
        access_level = f"Are all users/users attributes in {pred} granted proper access rights derived from the access control policy given?"
        pml_format = f"Does the generated configuration correctly follow Policy Machine Language format?"


        with dspy.context(lm=model):

            raw_users_response = self.evaluator(criterion= users,
                                            natural_language_description = description, 
                                            gold_configuration = gold_config,
                                            generated_configuration=gen_config)
            raw_ground_truth_response = self.evaluator(criterion= ground_truth,
                                            natural_language_description = description, 
                                        gold_configuration = gold_config,
                                        generated_configuration=gen_config)
            raw_access_response = self.evaluator(criterion= access_level,
                                        natural_language_description = description, 
                                        gold_configuration = gold_config,
                                        generated_configuration=gen_config)
            raw_format_response = self.evaluator(criterion= pml_format,
                                        natural_language_description = description, 
                                        gold_configuration = gold_config,
                                        generated_configuration=gen_config)
            
        
        with dspy.context(lm=model):
                user_rating = self.rating_parser(raw_rating_response=str(raw_users_response.feedback)).rating
                access_rating = self.rating_parser(raw_rating_response=str(raw_access_response.feedback)).rating
                ground_truth_rating = self.rating_parser(raw_rating_response=str(raw_ground_truth_response.feedback)).rating
                format_rating = self.rating_parser(raw_rating_response=str(raw_format_response.feedback)).rating
                

        user_rating = float(user_rating[-1:])
        access_rating = float(access_rating[-1:])
        ground_truth_rating = float(ground_truth_rating[-1:])
        format_rating = float(format_rating[-1:])

        total = user_rating + access_rating + ground_truth_rating + format_rating
        
        return total 



In [47]:
testset

[Example({'description': 'Access control for protecting sales data, inventory, and customer information. Sales\nManagers have full control over sales data, allowing them to read, write, create, and delete.\nInventory Managers have full control over inventory data, allowing them to read, write,\ncreate, and delete. Customer Service can read and write customer information but are\nrestricted from creating or deleting to prevent accidental data loss or breaches. Auditors\nhave read-only access to all types of data, ensuring they can perform audits without\nmodifying any data.', 'configuration': 'create policy class "SalesManagementPolicy"; create user attribute "SalesManagers" in\n["SalesManagementPolicy"]; create user attribute "InventoryManagers" in\n["SalesManagementPolicy"]; create user attribute "CustomerService" in\n["SalesManagementPolicy"]; create user attribute "Auditors" in\n["SalesManagementPolicy"]; create object attribute "SalesData" in\n["SalesManagementPolicy"]; create obje

In [48]:
question = testset[1].description

In [49]:
test_run_prompt = question

In [50]:
llm_metric = RAGMetric()
test_example = devset[0]
predicted = rag_model(test_run_prompt)
llm_metric_rating = llm_metric(test_example, predicted)
print(llm_metric_rating)

8.0


In [51]:
test2 = devset[1]
print(f"Policy: {test_run_prompt}\n")
print(f"Uncompiled RAG answer: {rag_model(test_run_prompt).configuration}")
print(f"RAG score: {llm_metric(test2, predicted)}")




Policy: Access control for protecting sales data, inventory, and customer information. Sales
Managers have full control over sales data, allowing them to read, write, create, and delete.
Bob is a sales manager. Inventory Managers have full control over inventory data, allowing
them to read, write, create, and delete. Alice is an inventory manager. Customer Service
can read and write customer information but are restricted from creating or deleting to
prevent accidental data loss or breaches. Auditors have read-only access to all types of
data, ensuring they can perform audits without modifying any data. Christine is an auditor.

Uncompiled RAG answer: Prediction(
    PML_statements='create policy class "sales_data";\ncreate policy class "inventory_data";\ncreate policy class "customer_information";\n\ncreate user attribute "sales_manager" in ["sales_data"];\ncreate user attribute "inventory_manager" in ["inventory_data"];\ncreate user attribute "customer_service" in ["customer_informat

In [52]:
devset[0]

Example({'description': 'In an IT company, different users have different permissions on several objects. Quality\nAssurance workers can print objects from UserDatabase; Cloud Specialists can move\nobjects in User Database. Cyber Security Experts can archive System Log objects. Jessica\noccupies ITCoordinator; Ashley is a member of CyberSecurityExpert; David is in\nQualityAssurance', 'configuration': 'create policy class "ITPolicyClass";create user attribute "ITCoordinator" in\n["ITPolicyClass"];create user attribute "CyberSecurityExpert" in ["ITPolicyClass"];create\nuser attribute "QualityAssurance" in ["ITPolicyClass"];create user attribute\n"CloudSpecialist" in ["ITPolicyClass"]; create object attribute "UserDatabase" in\n["ITPolicyClass"];create object attribute "SystemLog" in ["ITPolicyClass"]; create user\n"Jessica" in ["ITCoordinator"];create user "Ashley" in ["CyberSecurityExpert"];create user\n"David" in ["QualityAssurance"]; associate "QualityAssurance" and "UserDatabase" wit

In [53]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

evaluate(RAG(), metric=llm_metric)

Average Metric: 44.0 / 3  (1466.7): 100%|██████████| 3/3 [00:36<00:00, 12.02s/it]


,description,example_configuration,context,pred_configuration,RAGMetric
0,"In an IT company, different users have different permissions on several objects. Quality Assurance workers can print objects from UserDatabase; Cloud Specialists can move objects...","create policy class ""ITPolicyClass"";create user attribute ""ITCoordinator"" in [""ITPolicyClass""];create user attribute ""CyberSecurityExpert"" in [""ITPolicyClass""];create user attribute ""QualityAssurance"" in [""ITPolicyClass""];create user attribute ""CloudSpecialist"" in [""ITPolicyClass""]; create...",['A resource access is the only way by which users shall gain access to resources. An administration access is the only way by which users...,"Prediction( PML_statements='create policy class ""ITCompany"";\n\ncreate user attribute ""QualityAssuranceWorker"" in [""ITCompany""];\ncreate user attribute ""CloudSpecialist"" in [""ITCompany""];\ncreate user attribute ""CyberSecurityExpert"" in [""ITCompany""];\ncreate user attribute ""ITCoordinator"" in [""ITCompany""];\n\ncreate...",✔️ [16.0]
1,"In a lawfirm, several members can do different actions on legal objects. Attorneys can read/write legal cases. Lead Attorneys can create and delete cases. Interns...","create policy class ""LawFirmPolicyClass""; create user attribute ""Attorneys"" in [""LawFirmPolicyClass""]; create user attribute ""LeadAttorneys"" in [""LawFirmPolicyClass""]; create user attribute ""Interns"" in [""LawFirmPolicyClass""]; create object attribute...","['\nFunctionally oriented terms\nTrusted entity - An entity in which confidence exists that it can be relied upon to behave as expected.', 'Functional entity - A...","Prediction( PML_statements='create policy class ""lawfirm"";\n\ncreate user attribute ""attorney"" in [""lawfirm""];\ncreate user attribute ""leadattorney"" in [""lawfirm""];\ncreate user attribute ""intern"" in [""lawfirm""];\n\ncreate object attribute ""legalcase"" in [""lawfirm""];\n\nassign...",✔️ [14.0]
2,"In a lawfirm, several members can do different actions on legal objects. Attorneys can read/write legal cases. Lead Attorneys can create and delete cases. Interns...","create policy class ""LawFirmPolicyClass""; create user attribute ""Attorneys"" in [""LawFirmPolicyClass""]; create user attribute ""LeadAttorneys"" in [""LawFirmPolicyClass""]; create user attribute ""Interns"" in [""LawFirmPolicyClass""]; create object attribute...","['\nFunctionally oriented terms\nTrusted entity - An entity in which confidence exists that it can be relied upon to behave as expected.', 'Functional entity - A...","Prediction( PML_statements='create policy class ""lawfirm"";\n\ncreate user attribute ""attorney"" in [""lawfirm""];\ncreate user attribute ""leadattorney"" in [""lawfirm""];\ncreate user attribute ""intern"" in [""lawfirm""];\n\ncreate object attribute ""legalcase"" in [""lawfirm""];\n\nassign...",✔️ [14.0]


1466.67

In [54]:
from dspy.teleprompt import BootstrapFewShot

#teacherLM = dspy.OpenAI(model="gpt-4", max_tokens=12_000, model_type='chat')
teleprompter = BootstrapFewShot(metric=llm_metric, max_bootstrapped_demos=2, max_rounds=1)

compiledRAG = teleprompter.compile(RAG(), trainset=trainset)

100%|██████████| 2/2 [00:14<00:00,  7.34s/it]


In [55]:
compiledRAG.save("compiledRAG.json")

In [56]:
compiledRAG = RAG()
compiledRAG.load("compiledRAG.json")

In [57]:
compiled_score = evaluate(compiledRAG, metric=llm_metric)
print(f"Complied score = {compiled_score}.\n")

Average Metric: 46.0 / 3  (1533.3): 100%|██████████| 3/3 [00:46<00:00, 15.39s/it]


,description,example_configuration,context,pred_configuration,RAGMetric
0,"In an IT company, different users have different permissions on several objects. Quality Assurance workers can print objects from UserDatabase; Cloud Specialists can move objects...","create policy class ""ITPolicyClass"";create user attribute ""ITCoordinator"" in [""ITPolicyClass""];create user attribute ""CyberSecurityExpert"" in [""ITPolicyClass""];create user attribute ""QualityAssurance"" in [""ITPolicyClass""];create user attribute ""CloudSpecialist"" in [""ITPolicyClass""]; create...",['A resource access is the only way by which users shall gain access to resources. An administration access is the only way by which users...,"Prediction( PML_statements='create policy class ""ITCompanyAccessControl""; \ncreate user attribute ""QualityAssurance"" in [""ITCompanyAccessControl""]; \ncreate user attribute ""CloudSpecialist"" in [""ITCompanyAccessControl""]; \ncreate user attribute ""CyberSecurityExpert"" in [""ITCompanyAccessControl""]; \ncreate user...",✔️ [20.0]
1,"In a lawfirm, several members can do different actions on legal objects. Attorneys can read/write legal cases. Lead Attorneys can create and delete cases. Interns...","create policy class ""LawFirmPolicyClass""; create user attribute ""Attorneys"" in [""LawFirmPolicyClass""]; create user attribute ""LeadAttorneys"" in [""LawFirmPolicyClass""]; create user attribute ""Interns"" in [""LawFirmPolicyClass""]; create object attribute...","['\nFunctionally oriented terms\nTrusted entity - An entity in which confidence exists that it can be relied upon to behave as expected.', 'Functional entity - A...","Prediction( PML_statements='create policy class ""LawFirmAccessControl""; \ncreate user attribute ""Attorney"" in [""LawFirmAccessControl""]; \ncreate user attribute ""LeadAttorney"" in [""LawFirmAccessControl""]; \ncreate user attribute ""Intern"" in [""LawFirmAccessControl""]; \ncreate object...",✔️ [11.0]
2,"In a lawfirm, several members can do different actions on legal objects. Attorneys can read/write legal cases. Lead Attorneys can create and delete cases. Interns...","create policy class ""LawFirmPolicyClass""; create user attribute ""Attorneys"" in [""LawFirmPolicyClass""]; create user attribute ""LeadAttorneys"" in [""LawFirmPolicyClass""]; create user attribute ""Interns"" in [""LawFirmPolicyClass""]; create object attribute...","['\nFunctionally oriented terms\nTrusted entity - An entity in which confidence exists that it can be relied upon to behave as expected.', 'Functional entity - A...","Prediction( PML_statements='create policy class ""lawfirm""; \ncreate user attribute ""Attorney"" in [""lawfirm""]; \ncreate user attribute ""LeadAttorney"" in [""lawfirm""]; \ncreate user attribute ""Intern"" in [""lawfirm""]; \ncreate object...",✔️ [15.0]


Complied score = 1533.33.



In [58]:
testset

[Example({'description': 'Access control for protecting sales data, inventory, and customer information. Sales\nManagers have full control over sales data, allowing them to read, write, create, and delete.\nInventory Managers have full control over inventory data, allowing them to read, write,\ncreate, and delete. Customer Service can read and write customer information but are\nrestricted from creating or deleting to prevent accidental data loss or breaches. Auditors\nhave read-only access to all types of data, ensuring they can perform audits without\nmodifying any data.', 'configuration': 'create policy class "SalesManagementPolicy"; create user attribute "SalesManagers" in\n["SalesManagementPolicy"]; create user attribute "InventoryManagers" in\n["SalesManagementPolicy"]; create user attribute "CustomerService" in\n["SalesManagementPolicy"]; create user attribute "Auditors" in\n["SalesManagementPolicy"]; create object attribute "SalesData" in\n["SalesManagementPolicy"]; create obje

In [59]:
for example in testset:
    description = example.description
    configuration = example.configuration
    response = compiledRAG(description)
    print(response.configuration)
    metric_pred = llm_metric(example, response)
    print(f"LLM Rating: {metric_pred}\n")

Prediction(
    PML_statements='create policy class "AccessControl"; \ncreate user attribute "SalesManager" in ["AccessControl"]; \ncreate user attribute "InventoryManager" in ["AccessControl"]; \ncreate user attribute "CustomerService" in ["AccessControl"]; \ncreate object attribute "SalesData" in ["AccessControl"]; \ncreate object attribute "InventoryData" in ["AccessControl"]; \ncreate object attribute "CustomerInformation" in ["AccessControl"]; \nassign "SalesManager" to ["InventoryManager", "CustomerService"]; \ncreate user "SalesManager1" in ["SalesManager"]; \ncreate user "InventoryManager1" in ["InventoryManager"]; \ncreate user "CustomerService1" in ["CustomerService"]; \nassociate "SalesManager" and "SalesData" with ["read", "write", "create", "delete"]; \nassociate "InventoryManager" and "InventoryData" with ["read", "write", "create", "delete"]; \nassociate "CustomerService" and "CustomerInformation" with ["read", "write"];'
)
LLM Rating: 12.0

Prediction(
    PML_statement

In [60]:
model.inspect_history(n=1)




Parse the rating from a string.

---

Follow the following format.

Raw Rating Response: The string that contains the rating in it.
Rating: An integer valued rating.

---

Raw Rating Response: 4
Rating: 4





'\n\n\nParse the rating from a string.\n\n---\n\nFollow the following format.\n\nRaw Rating Response: The string that contains the rating in it.\nRating: An integer valued rating.\n\n---\n\nRaw Rating Response: 4\nRating:\x1b 4\x1b\n\n\n'